<a href="https://colab.research.google.com/github/amimulhasan/Deep-Learning/blob/main/CNN_%2B_ViT_%2B_GRU_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download mohammadhossein77/brain-tumors-dataset

Dataset URL: https://www.kaggle.com/datasets/mohammadhossein77/brain-tumors-dataset
License(s): CC0-1.0
 91% 202M/221M [00:00<00:00, 581MB/s] 
100% 221M/221M [00:00<00:00, 620MB/s]


In [4]:
import zipfile

zip_path = '/content/brain-tumors-dataset.zip'
extract_to = 'brain_tumor_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Unzipping completed!")

Unzipping completed!


In [5]:
batch_size = 32
img_size = (128, 128)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/brain_tumor_data',
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/brain_tumor_data',
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Classes:", class_names)


Found 21672 files belonging to 1 classes.
Using 17338 files for training.
Found 21672 files belonging to 1 classes.
Using 4334 files for validation.
Classes: ['Data']


In [6]:
class ViTBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout_rate=0.1):
        super(ViTBlock, self).__init__()
        self.norm1 = layers.LayerNormalization()
        self.attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.drop1 = layers.Dropout(dropout_rate)

        self.norm2 = layers.LayerNormalization()
        self.mlp = tf.keras.Sequential([
            layers.Dense(mlp_dim, activation='relu'),
            layers.Dropout(dropout_rate),
            layers.Dense(embed_dim)
        ])
        self.drop2 = layers.Dropout(dropout_rate)

    def call(self, x):
        attn_out = self.attn(x, x)
        x = x + self.drop1(attn_out)
        x = self.norm1(x)
        mlp_out = self.mlp(x)
        x = x + self.drop2(mlp_out)
        return self.norm2(x)


In [7]:
def build_cnn_vit_gru(input_shape=(128, 128, 3), num_classes=4):
    inputs = layers.Input(shape=input_shape)

    # CNN base
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)

    # Flatten spatial dims to sequence
    x = layers.Reshape((-1, 128))(x)  # shape = (batch, time_steps, features)

    # ViT Block
    x = ViTBlock(embed_dim=128, num_heads=4, mlp_dim=256)(x)

    # GRU
    x = layers.Bidirectional(layers.GRU(64, return_sequences=False))(x)

    # Output
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model


In [ ]:
model = build_cnn_vit_gru()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

history = model.fit(train_ds, validation_data=val_ds, epochs=10)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 256, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vi_t_block (ViTBlock)           │ (None, 256, 128)       │       330,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 506,500 (1.93 MB)

 Trainable params: 506,500 (1.93 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 1557s 3s/step - accuracy: 0.9925 - loss: 0.0181 - val_accuracy: 1.0000 - val_loss: 1.3427e-06
Epoch 2/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 1628s 3s/step - accuracy: 1.0000 - loss: 1.2993e-06 - val_accuracy: 1.0000 - val_loss: 2.3968e-07
Epoch 3/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 1565s 3s/step - accuracy: 1.0000 - loss: 3.4542e-07 - val_accuracy: 1.0000 - val_loss: 1.1929e-07
Epoch 4/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 1468s 3s/step - accuracy: 1.0000 - loss: 1.9925e-07 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 1480s 3s/step - accuracy: 1.0000 - loss: 2.1290e-08 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 1577s 3s/step - accuracy: 1.0000 - loss: 5.4291e-10 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
542/542 ━━━━━━━━━━━━━━━━━━━━ 1558s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
542/542 ━━━━━━━━━━━━━━━━━━━

In [ ]:
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('CNN+ViT+GRU Model Accuracy')
plt.grid(True)
plt.show()
